In [1]:
import chess
import random
import copy

current_state = ('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1')
board = chess.Board()
win_outcomes = []
total_outcomes = []
ratios = []
iterations_per_option = 80
cpu_colour = 'b'
end = False

In [2]:
grid = []
Battackers = []
Wattackers = []

# these are number of pieces

BrooksAlive = []
BknightsAlive = []
BbishopsAlive = []
BqueenAlive = []
BkingAlive = []
BpawnsAlive = []

WrooksAlive = []
WknightsAlive = []
WbishopsAlive = []
WqueenAlive = []
WkingAlive = []
WpawnsAlive = []

# these are number of vulnerabilities to these pieces

BrooksVulner = []
BknightsVulner = []
BbishopsVulner = []
BqueenVulner = []
BkingVulner = []
BpawnsVulner = []

WrooksVulner = []
WknightsVulner = []
WbishopsVulner = []
WqueenVulner = []
WkingVulner = []
WpawnsVulner = []

# these are number of defenses to these pieces

BrooksDefense = []
BknightsDefense = []
BbishopsDefense = []
BqueenDefense = []
BkingDefense = []
BpawnsDefense = []

WrooksDefense = []
WknightsDefense = []
WbishopsDefense = []
WqueenDefense = []
WkingDefense = []
WpawnsDefense = []

# these are number of directions the pieces can move

BrooksDirection = []
BknightsDirection = []
BbishopsDirection = []
BqueenDirection = []
BkingDirection = []
BpawnsDirection = []

WrooksDirection = []
WknightsDirection = []
WbishopsDirection = []
WqueenDirection = []
WkingDirection = []
WpawnsDirection = []

#these are raw number of safe spaces for white and black on their respective sides

Bsafe = []
Wsafe = []

#this is the score based off of Piece Square Tables

PSTscore = []

#these are the valuations for each of the moves
moveValues = []

#these are the index of the 5 best valued moves
top5 = []

In [3]:
game_board = chess.Board()
Brook = game_board.piece_at(chess.A8)
Bknight = game_board.piece_at(chess.B8)
Bbishop = game_board.piece_at(chess.C8)
Bqueen = game_board.piece_at(chess.D8)
Bking = game_board.piece_at(chess.E8)
Bpawn = game_board.piece_at(chess.A7)

Wrook = game_board.piece_at(chess.A1)
Wknight = game_board.piece_at(chess.B1)
Wbishop = game_board.piece_at(chess.C1)
Wqueen = game_board.piece_at(chess.D1)
Wking = game_board.piece_at(chess.E1)
Wpawn = game_board.piece_at(chess.A2)

Empty = game_board.piece_at(chess.B4)

In [4]:
pawnTable = [0,  0,  0,  0,  0,  0,  0,  0,
            50, 50, 50, 50, 50, 50, 50, 50,
            10, 10, 20, 30, 30, 20, 10, 10,
             5,  5, 10, 27, 27, 10,  5,  5,
             0,  0,  0, 25, 25,  0,  0,  0,
             5, -5,-10,  0,  0,-10, -5,  5,
             5, 10, 10,-25,-25, 10, 10,  5,
             0,  0,  0,  0,  0,  0,  0,  0]

knightTable = [ -50,-40,-30,-30,-30,-30,-40,-50,
                -40,-20,  0,  0,  0,  0,-20,-40,
                -30,  0, 10, 15, 15, 10,  0,-30,
                -30,  5, 15, 20, 20, 15,  5,-30,
                -30,  0, 15, 20, 20, 15,  0,-30,
                -30,  5, 10, 15, 15, 10,  5,-30,
                -40,-20,  0,  5,  5,  0,-20,-40,
                -50,-40,-20,-30,-30,-20,-40,-50]

bishopTable = [ -20,-10,-10,-10,-10,-10,-10,-20,
                -10,  0,  0,  0,  0,  0,  0,-10,
                -10,  0,  5, 10, 10,  5,  0,-10,
                -10,  5,  5, 10, 10,  5,  5,-10,
                -10,  0, 10, 10, 10, 10,  0,-10,
                -10, 10, 10, 10, 10, 10, 10,-10,
                -10,  5,  0,  0,  0,  0,  5,-10,
                -20,-10,-40,-10,-10,-40,-10,-20]

kingTableEarly = [-30, -40, -40, -50, -50, -40, -40, -30,
                  -30, -40, -40, -50, -50, -40, -40, -30,
                  -30, -40, -40, -50, -50, -40, -40, -30,
                  -30, -40, -40, -50, -50, -40, -40, -30,
                  -20, -30, -30, -40, -40, -30, -30, -20,
                  -10, -20, -20, -20, -20, -20, -20, -10, 
                   20,  20,   0,   0,   0,   0,  20,  20,
                   20,  30,  10,   0,   0,  10,  30,  20]

kingTableLate = [-50,-40,-30,-20,-20,-30,-40,-50,
                -30,-20,-10,  0,  0,-10,-20,-30,
                -30,-10, 20, 30, 30, 20,-10,-30,
                -30,-10, 30, 40, 40, 30,-10,-30,
                -30,-10, 30, 40, 40, 30,-10,-30,
                -30,-10, 20, 30, 30, 20,-10,-30,
                -30,-30,  0,  0,  0,  0,-30,-30,
                -50,-30,-30,-30,-30,-30,-30,-50]

In [5]:
def emptyGrid():
    grid.clear()
    for i in range(8):
        grid.append([])

In [6]:
def emptyWattackers():
    Wattackers.clear()
    for i in range(8):
        Wattackers.append([])

In [7]:
def emptyBattackers():
    Battackers.clear()
    for i in range(8):
        Battackers.append([])

In [8]:
def updateGridInfo():
    grid[0].append(game_board.piece_at(chess.A1))
    grid[0].append(game_board.piece_at(chess.B1))
    grid[0].append(game_board.piece_at(chess.C1))
    grid[0].append(game_board.piece_at(chess.D1))
    grid[0].append(game_board.piece_at(chess.E1))
    grid[0].append(game_board.piece_at(chess.F1))
    grid[0].append(game_board.piece_at(chess.G1))
    grid[0].append(game_board.piece_at(chess.H1))
    grid[1].append(game_board.piece_at(chess.A2))
    grid[1].append(game_board.piece_at(chess.B2))
    grid[1].append(game_board.piece_at(chess.C2))
    grid[1].append(game_board.piece_at(chess.D2))
    grid[1].append(game_board.piece_at(chess.E2))
    grid[1].append(game_board.piece_at(chess.F2))
    grid[1].append(game_board.piece_at(chess.G2))
    grid[1].append(game_board.piece_at(chess.H2))
    grid[2].append(game_board.piece_at(chess.A3))
    grid[2].append(game_board.piece_at(chess.B3))
    grid[2].append(game_board.piece_at(chess.C3))
    grid[2].append(game_board.piece_at(chess.D3))
    grid[2].append(game_board.piece_at(chess.E3))
    grid[2].append(game_board.piece_at(chess.F3))
    grid[2].append(game_board.piece_at(chess.G3))
    grid[2].append(game_board.piece_at(chess.H3))
    grid[3].append(game_board.piece_at(chess.A4))
    grid[3].append(game_board.piece_at(chess.B4))
    grid[3].append(game_board.piece_at(chess.C4))
    grid[3].append(game_board.piece_at(chess.D4))
    grid[3].append(game_board.piece_at(chess.E4))
    grid[3].append(game_board.piece_at(chess.F4))
    grid[3].append(game_board.piece_at(chess.G4))
    grid[3].append(game_board.piece_at(chess.H4))
    grid[4].append(game_board.piece_at(chess.A5))
    grid[4].append(game_board.piece_at(chess.B5))
    grid[4].append(game_board.piece_at(chess.C5))
    grid[4].append(game_board.piece_at(chess.D5))
    grid[4].append(game_board.piece_at(chess.E5))
    grid[4].append(game_board.piece_at(chess.F5))
    grid[4].append(game_board.piece_at(chess.G5))
    grid[4].append(game_board.piece_at(chess.H5))
    grid[5].append(game_board.piece_at(chess.A6))
    grid[5].append(game_board.piece_at(chess.B6))
    grid[5].append(game_board.piece_at(chess.C6))
    grid[5].append(game_board.piece_at(chess.D6))
    grid[5].append(game_board.piece_at(chess.E6))
    grid[5].append(game_board.piece_at(chess.F6))
    grid[5].append(game_board.piece_at(chess.G6))
    grid[5].append(game_board.piece_at(chess.H6))
    grid[6].append(game_board.piece_at(chess.A7))
    grid[6].append(game_board.piece_at(chess.B7))
    grid[6].append(game_board.piece_at(chess.C7))
    grid[6].append(game_board.piece_at(chess.D7))
    grid[6].append(game_board.piece_at(chess.E7))
    grid[6].append(game_board.piece_at(chess.F7))
    grid[6].append(game_board.piece_at(chess.G7))
    grid[6].append(game_board.piece_at(chess.H7))
    grid[7].append(game_board.piece_at(chess.A8))
    grid[7].append(game_board.piece_at(chess.B8))
    grid[7].append(game_board.piece_at(chess.C8))
    grid[7].append(game_board.piece_at(chess.D8))
    grid[7].append(game_board.piece_at(chess.E8))
    grid[7].append(game_board.piece_at(chess.F8))
    grid[7].append(game_board.piece_at(chess.G8))
    grid[7].append(game_board.piece_at(chess.H8))

In [9]:
def updateWAttackersInfo():
    Wattackers[0].append(game_board.attackers(chess.WHITE, chess.A1))
    Wattackers[0].append(game_board.attackers(chess.WHITE, chess.B1))
    Wattackers[0].append(game_board.attackers(chess.WHITE, chess.C1))
    Wattackers[0].append(game_board.attackers(chess.WHITE, chess.D1))
    Wattackers[0].append(game_board.attackers(chess.WHITE, chess.E1))
    Wattackers[0].append(game_board.attackers(chess.WHITE, chess.F1))
    Wattackers[0].append(game_board.attackers(chess.WHITE, chess.G1))
    Wattackers[0].append(game_board.attackers(chess.WHITE, chess.H1))
    Wattackers[1].append(game_board.attackers(chess.WHITE, chess.A2))
    Wattackers[1].append(game_board.attackers(chess.WHITE, chess.B2))
    Wattackers[1].append(game_board.attackers(chess.WHITE, chess.C2))
    Wattackers[1].append(game_board.attackers(chess.WHITE, chess.D2))
    Wattackers[1].append(game_board.attackers(chess.WHITE, chess.E2))
    Wattackers[1].append(game_board.attackers(chess.WHITE, chess.F2))
    Wattackers[1].append(game_board.attackers(chess.WHITE, chess.G2))
    Wattackers[1].append(game_board.attackers(chess.WHITE, chess.H2))
    Wattackers[2].append(game_board.attackers(chess.WHITE, chess.A3))
    Wattackers[2].append(game_board.attackers(chess.WHITE, chess.B3))
    Wattackers[2].append(game_board.attackers(chess.WHITE, chess.C3))
    Wattackers[2].append(game_board.attackers(chess.WHITE, chess.D3))
    Wattackers[2].append(game_board.attackers(chess.WHITE, chess.E3))
    Wattackers[2].append(game_board.attackers(chess.WHITE, chess.F3))
    Wattackers[2].append(game_board.attackers(chess.WHITE, chess.G3))
    Wattackers[2].append(game_board.attackers(chess.WHITE, chess.H3))
    Wattackers[3].append(game_board.attackers(chess.WHITE, chess.A4))
    Wattackers[3].append(game_board.attackers(chess.WHITE, chess.B4))
    Wattackers[3].append(game_board.attackers(chess.WHITE, chess.C4))
    Wattackers[3].append(game_board.attackers(chess.WHITE, chess.D4))
    Wattackers[3].append(game_board.attackers(chess.WHITE, chess.E4))
    Wattackers[3].append(game_board.attackers(chess.WHITE, chess.F4))
    Wattackers[3].append(game_board.attackers(chess.WHITE, chess.G4))
    Wattackers[3].append(game_board.attackers(chess.WHITE, chess.H4))
    Wattackers[4].append(game_board.attackers(chess.WHITE, chess.A5))
    Wattackers[4].append(game_board.attackers(chess.WHITE, chess.B5))
    Wattackers[4].append(game_board.attackers(chess.WHITE, chess.C5))
    Wattackers[4].append(game_board.attackers(chess.WHITE, chess.D5))
    Wattackers[4].append(game_board.attackers(chess.WHITE, chess.E5))
    Wattackers[4].append(game_board.attackers(chess.WHITE, chess.F5))
    Wattackers[4].append(game_board.attackers(chess.WHITE, chess.G5))
    Wattackers[4].append(game_board.attackers(chess.WHITE, chess.H5))
    Wattackers[5].append(game_board.attackers(chess.WHITE, chess.A6))
    Wattackers[5].append(game_board.attackers(chess.WHITE, chess.B6))
    Wattackers[5].append(game_board.attackers(chess.WHITE, chess.C6))
    Wattackers[5].append(game_board.attackers(chess.WHITE, chess.D6))
    Wattackers[5].append(game_board.attackers(chess.WHITE, chess.E6))
    Wattackers[5].append(game_board.attackers(chess.WHITE, chess.F6))
    Wattackers[5].append(game_board.attackers(chess.WHITE, chess.G6))
    Wattackers[5].append(game_board.attackers(chess.WHITE, chess.H6))
    Wattackers[6].append(game_board.attackers(chess.WHITE, chess.A7))
    Wattackers[6].append(game_board.attackers(chess.WHITE, chess.B7))
    Wattackers[6].append(game_board.attackers(chess.WHITE, chess.C7))
    Wattackers[6].append(game_board.attackers(chess.WHITE, chess.D7))
    Wattackers[6].append(game_board.attackers(chess.WHITE, chess.E7))
    Wattackers[6].append(game_board.attackers(chess.WHITE, chess.F7))
    Wattackers[6].append(game_board.attackers(chess.WHITE, chess.G7))
    Wattackers[6].append(game_board.attackers(chess.WHITE, chess.H7))
    Wattackers[7].append(game_board.attackers(chess.WHITE, chess.A8))
    Wattackers[7].append(game_board.attackers(chess.WHITE, chess.B8))
    Wattackers[7].append(game_board.attackers(chess.WHITE, chess.C8))
    Wattackers[7].append(game_board.attackers(chess.WHITE, chess.D8))
    Wattackers[7].append(game_board.attackers(chess.WHITE, chess.E8))
    Wattackers[7].append(game_board.attackers(chess.WHITE, chess.F8))
    Wattackers[7].append(game_board.attackers(chess.WHITE, chess.G8))
    Wattackers[7].append(game_board.attackers(chess.WHITE, chess.H8))

In [10]:
def updateBAttackersInfo():
    Battackers[0].append(game_board.attackers(chess.BLACK, chess.A1))
    Battackers[0].append(game_board.attackers(chess.BLACK, chess.B1))
    Battackers[0].append(game_board.attackers(chess.BLACK, chess.C1))
    Battackers[0].append(game_board.attackers(chess.BLACK, chess.D1))
    Battackers[0].append(game_board.attackers(chess.BLACK, chess.E1))
    Battackers[0].append(game_board.attackers(chess.BLACK, chess.F1))
    Battackers[0].append(game_board.attackers(chess.BLACK, chess.G1))
    Battackers[0].append(game_board.attackers(chess.BLACK, chess.H1))
    Battackers[1].append(game_board.attackers(chess.BLACK, chess.A2))
    Battackers[1].append(game_board.attackers(chess.BLACK, chess.B2))
    Battackers[1].append(game_board.attackers(chess.BLACK, chess.C2))
    Battackers[1].append(game_board.attackers(chess.BLACK, chess.D2))
    Battackers[1].append(game_board.attackers(chess.BLACK, chess.E2))
    Battackers[1].append(game_board.attackers(chess.BLACK, chess.F2))
    Battackers[1].append(game_board.attackers(chess.BLACK, chess.G2))
    Battackers[1].append(game_board.attackers(chess.BLACK, chess.H2))
    Battackers[2].append(game_board.attackers(chess.BLACK, chess.A3))
    Battackers[2].append(game_board.attackers(chess.BLACK, chess.B3))
    Battackers[2].append(game_board.attackers(chess.BLACK, chess.C3))
    Battackers[2].append(game_board.attackers(chess.BLACK, chess.D3))
    Battackers[2].append(game_board.attackers(chess.BLACK, chess.E3))
    Battackers[2].append(game_board.attackers(chess.BLACK, chess.F3))
    Battackers[2].append(game_board.attackers(chess.BLACK, chess.G3))
    Battackers[2].append(game_board.attackers(chess.BLACK, chess.H3))
    Battackers[3].append(game_board.attackers(chess.BLACK, chess.A4))
    Battackers[3].append(game_board.attackers(chess.BLACK, chess.B4))
    Battackers[3].append(game_board.attackers(chess.BLACK, chess.C4))
    Battackers[3].append(game_board.attackers(chess.BLACK, chess.D4))
    Battackers[3].append(game_board.attackers(chess.BLACK, chess.E4))
    Battackers[3].append(game_board.attackers(chess.BLACK, chess.F4))
    Battackers[3].append(game_board.attackers(chess.BLACK, chess.G4))
    Battackers[3].append(game_board.attackers(chess.BLACK, chess.H4))
    Battackers[4].append(game_board.attackers(chess.BLACK, chess.A5))
    Battackers[4].append(game_board.attackers(chess.BLACK, chess.B5))
    Battackers[4].append(game_board.attackers(chess.BLACK, chess.C5))
    Battackers[4].append(game_board.attackers(chess.BLACK, chess.D5))
    Battackers[4].append(game_board.attackers(chess.BLACK, chess.E5))
    Battackers[4].append(game_board.attackers(chess.BLACK, chess.F5))
    Battackers[4].append(game_board.attackers(chess.BLACK, chess.G5))
    Battackers[4].append(game_board.attackers(chess.BLACK, chess.H5))
    Battackers[5].append(game_board.attackers(chess.BLACK, chess.A6))
    Battackers[5].append(game_board.attackers(chess.BLACK, chess.B6))
    Battackers[5].append(game_board.attackers(chess.BLACK, chess.C6))
    Battackers[5].append(game_board.attackers(chess.BLACK, chess.D6))
    Battackers[5].append(game_board.attackers(chess.BLACK, chess.E6))
    Battackers[5].append(game_board.attackers(chess.BLACK, chess.F6))
    Battackers[5].append(game_board.attackers(chess.BLACK, chess.G6))
    Battackers[5].append(game_board.attackers(chess.BLACK, chess.H6))
    Battackers[6].append(game_board.attackers(chess.BLACK, chess.A7))
    Battackers[6].append(game_board.attackers(chess.BLACK, chess.B7))
    Battackers[6].append(game_board.attackers(chess.BLACK, chess.C7))
    Battackers[6].append(game_board.attackers(chess.BLACK, chess.D7))
    Battackers[6].append(game_board.attackers(chess.BLACK, chess.E7))
    Battackers[6].append(game_board.attackers(chess.BLACK, chess.F7))
    Battackers[6].append(game_board.attackers(chess.BLACK, chess.G7))
    Battackers[6].append(game_board.attackers(chess.BLACK, chess.H7))
    Battackers[7].append(game_board.attackers(chess.BLACK, chess.A8))
    Battackers[7].append(game_board.attackers(chess.BLACK, chess.B8))
    Battackers[7].append(game_board.attackers(chess.BLACK, chess.C8))
    Battackers[7].append(game_board.attackers(chess.BLACK, chess.D8))
    Battackers[7].append(game_board.attackers(chess.BLACK, chess.E8))
    Battackers[7].append(game_board.attackers(chess.BLACK, chess.F8))
    Battackers[7].append(game_board.attackers(chess.BLACK, chess.G8))
    Battackers[7].append(game_board.attackers(chess.BLACK, chess.H8))

In [11]:
def countAlive ():
    BrooksFound = 0
    BknightsFound = 0
    BbishopsFound = 0
    BqueenFound = 0
    BkingFound = 0
    BpawnsFound = 0

    WrooksFound = 0
    WknightsFound = 0
    WbishopsFound = 0
    WqueenFound = 0
    WkingFound = 0
    WpawnsFound = 0
    
    for i in range(8):
        for j in range(8):
            current = grid[i][j]
            if current==Brook:
                BrooksFound +=1
            elif current==Bknight:
                BknightsFound +=1
            elif current==Bbishop:
                BbishopsFound +=1
            elif current==Bqueen:
                BqueenFound =1
            elif current==Bking:
                BkingFound =1
            elif current==Bpawn:
                BpawnsFound +=1
            elif current==Wrook:
                WrooksFound +=1
            elif current==Wknight:
                WknightsFound +=1
            elif current==Wbishop:
                WbishopsFound +=1
            elif current==Wqueen:
                WqueenFound =1
            elif current==Wking:
                WkingFound =1
            elif current==Wpawn:
                WpawnsFound +=1
    
    BrooksAlive.append(BrooksFound)
    BknightsAlive.append(BknightsFound)
    BbishopsAlive.append(BbishopsFound)
    BqueenAlive.append(BqueenFound)
    BkingAlive.append(BkingFound)
    BpawnsAlive.append(BpawnsFound)
    
    WrooksAlive.append(WrooksFound)
    WknightsAlive.append(WknightsFound)
    WbishopsAlive.append(WbishopsFound)
    WqueenAlive.append(WqueenFound)
    WkingAlive.append(WkingFound)
    WpawnsAlive.append(WpawnsFound)

In [12]:
def updateVulDefInfo():
    
    BrooksVulnerTemp = 0
    BrooksDefenseTemp = 0
    
    BknightsVulnerTemp=0
    BknightsDefenseTemp=0
    
    BbishopsVulnerTemp=0
    BbishopsDefenseTemp=0      
    
    BqueenVulnerTemp=0
    BqueenDefenseTemp=0
    
    BkingVulnerTemp=0
    BkingDefenseTemp=0
    
    BpawnsVulnerTemp=0
    BpawnsDefenseTemp=0
    
    WrooksVulnerTemp=0
    WrooksDefenseTemp=0
    
    WknightsVulnerTemp=0
    WknightsDefenseTemp=0
            
    WbishopsVulnerTemp=0
    WbishopsDefenseTemp=0
            
    WqueenVulnerTemp=0
    WqueenDefenseTemp=0
            
    WkingVulnerTemp=0
    WkingDefenseTemp=0
            
    WpawnsVulnerTemp=0
    WpawnsDefenseTemp=0
    
    for i in range(8):
        for j in range(8):
            current = grid[i][j]
            Wopenings = len(Wattackers[i][j])
            Bopenings = len(Battackers[i][j])
            freeWattacks = Wopenings - Bopenings
            freeBattacks = Bopenings - Wopenings
            if freeWattacks<0:
                freeWattacks = 0
            if freeBattacks<0:
                freeBattacks = 0
            
            defendWattacks = Wopenings - freeWattacks
            defendBattacks = Bopenings - freeBattacks
            
            if current==Brook:
                BrooksVulnerTemp += freeWattacks
                BrooksDefenseTemp +=defendWattacks
            elif current==Bknight:
                BknightsVulnerTemp += freeWattacks
                BknightsDefenseTemp +=defendWattacks
                
            elif current==Bbishop:
                BbishopsVulnerTemp += freeWattacks
                BbishopsDefenseTemp +=defendWattacks 
            
            elif current==Bqueen:
                BqueenVulnerTemp += freeWattacks
                BqueenDefenseTemp +=defendWattacks 
            
            elif current==Bking:
                BkingVulnerTemp += freeWattacks
                BkingDefenseTemp +=defendWattacks
            
            elif current==Bpawn:
                BpawnsVulnerTemp += freeWattacks
                BpawnsDefenseTemp +=defendWattacks
            
            elif current==Wrook:
                WrooksVulnerTemp += freeBattacks
                WrooksDefenseTemp +=defendBattacks
            
            elif current==Wknight:
                WknightsVulnerTemp += freeBattacks
                WknightsDefenseTemp +=defendBattacks
            
            elif current==Wbishop:
                WbishopsVulnerTemp += freeBattacks
                WbishopsDefenseTemp +=defendBattacks
            
            elif current==Wqueen:
                WqueenVulnerTemp += freeBattacks
                WqueenDefenseTemp +=defendBattacks
            
            elif current==Wking:
                WkingVulnerTemp += freeBattacks
                WkingDefenseTemp +=defendBattacks
            
            elif current==Wpawn:
                WpawnsVulnerTemp += freeBattacks
                WpawnsDefenseTemp +=defendBattacks
    
    BrooksVulner.append(BrooksVulnerTemp)
    BrooksDefense.append(BrooksDefenseTemp)
    
    BknightsVulner.append(BknightsVulnerTemp)
    BknightsDefense.append(BknightsDefenseTemp)
    
    BbishopsVulner.append(BbishopsVulnerTemp)
    BbishopsDefense.append(BbishopsDefenseTemp)      
    
    BqueenVulner.append(BqueenVulnerTemp)
    BqueenDefense.append(BqueenDefenseTemp)
    
    BkingVulner.append(BkingVulnerTemp)
    BkingDefense.append(BkingDefenseTemp)
    
    BpawnsVulner.append(BpawnsVulnerTemp)
    BpawnsDefense.append(BpawnsDefenseTemp)
    
    WrooksVulner.append(WrooksVulnerTemp)
    WrooksDefense.append(WrooksDefenseTemp)
    
    WknightsVulner.append(WknightsVulnerTemp)
    WknightsDefense.append(WknightsDefenseTemp)
            
    WbishopsVulner.append(WbishopsVulnerTemp)
    WbishopsDefense.append(WbishopsDefenseTemp)
            
    WqueenVulner.append(WqueenVulnerTemp)
    WqueenDefense.append(WqueenDefenseTemp)
            
    WkingVulner.append(WkingVulnerTemp)
    WkingDefense.append(WkingDefenseTemp)
            
    WpawnsVulner.append(WpawnsVulnerTemp)
    WpawnsDefense.append(WpawnsDefenseTemp)

In [13]:
def rangeOfMotion():
    
    BrooksDirectionTemp = 0
    
    BknightsDirectionTemp=0
    
    BbishopsDirectionTemp=0    
    
    BqueenDirectionTemp=0
    
    BkingDirectionTemp=0
    
    BpawnsDirectionTemp=0
    
    
    WrooksDirectionTemp=0
    
    WknightsDirectionTemp=0
            
    WbishopsDirectionTemp=0
            
    WqueenDirectionTemp=0
            
    WkingDirectionTemp=0
            
    WpawnsDirectionTemp=0
    
    for i in range(8):
        for j in range(8):
            current = grid[i][j]
            Wup = 0
            Bup = 0
            if i<7 and grid[i+1][j]==Empty:
                if len(Battackers[i+1][j])==0:
                    Wup=1
                elif len(Wattackers[i+1][j])==0:
                    Bup=1
            
            Wdown = 0
            Bdown = 0
            if i>0 and grid[i-1][j]==Empty:
                if len(Battackers[i-1][j])==0:
                    Wdown=1
                elif len(Wattackers[i-1][j])==0:
                    Bdown=1
            Wleft = 0
            Bleft = 0
            if j>0 and grid[i][j-1]==Empty:
                if len(Battackers[i][j-1])==0:
                    Wleft=1
                elif len(Wattackers[i][j-1])==0:
                    Bleft=1
            Wright = 0
            Bright = 0
            if j<7 and grid[i][j+1]==Empty:
                if len(Battackers[i][j+1])==0:
                    Wright=1
                elif len(Wattackers[i][j+1])==0:
                    Bright=1
            Wtldiag = 0
            Btldiag = 0
            if i<7 and j>0 and grid[i+1][j-1]==Empty:
                if len(Battackers[i+1][j-1])==0:
                    Wtldiag=1
                elif len(Wattackers[i+1][j-1])==0:
                    Btldiag=1
            Wtrdiag = 0
            Btrdiag = 0
            if i<7 and j<7 and grid[i+1][j+1]==Empty:
                if len(Battackers[i+1][j+1])==0:
                    Wtrdiag=1
                elif len(Wattackers[i+1][j+1])==0:
                    Btrdiag=1
            Wbldiag = 0
            Bbldiag = 0
            if i>0 and j>0 and grid[i-1][j-1]==Empty:
                if len(Battackers[i-1][j-1])==0:
                    Wbldiag=1
                elif len(Wattackers[i-1][j-1])==0:
                    Bbldiag=1
            Wbrdiag = 0
            Bbrdiag = 0
            if i>0 and j<7 and grid[i-1][j+1]==Empty:
                if len(Battackers[i-1][j+1])==0:
                    Wbrdiag=1
                elif len(Wattackers[i-1][j+1])==0:
                    Bbrdiag=1
            
            Wtlhorse = 0
            Btlhorse = 0
            if i<6 and j>0 and grid[i+2][j-1]==Empty:
                if len(Battackers[i+2][j-1])==0:
                    Wtlhorse=1
                elif len(Wattackers[i+2][j-1])==0:
                    Btlhorse=1
                         
            Wtrhorse = 0
            Btrhorse = 0
            if i<6 and j<7 and grid[i+2][j+1]==Empty:
                if len(Battackers[i+2][j+1])==0:
                    Wtrhorse=1
                elif len(Wattackers[i+2][j+1])==0:
                    Btrhorse=1
            
            Wlthorse = 0
            Blthorse = 0
            if i<7 and j>1 and grid[i+1][j-2]==Empty:
                if len(Battackers[i+1][j-2])==0:
                    Wlthorse=1
                elif len(Wattackers[i+1][j-2])==0:
                    Blthorse=1
                         
            Wlbhorse = 0
            Blbhorse = 0
            if i>0 and j>1 and grid[i-1][j-2]==Empty:
                if len(Battackers[i-1][j-2])==0:
                    Wlbhorse=1
                elif len(Wattackers[i-1][j-2])==0:
                    Blbhorse=1
                         
            Wblhorse = 0
            Bblhorse = 0
            if i>1 and j>0 and grid[i-2][j-1]==Empty:
                if len(Battackers[i-2][j-1])==0:
                    Wblhorse=1
                elif len(Wattackers[i-2][j-1])==0:
                    Bblhorse=1
                         
            Wbrhorse = 0
            Bbrhorse = 0
            if i>1 and j<7 and grid[i-2][j+1]==Empty:
                if len(Battackers[i-2][j+1])==0:
                    Wbrhorse=1
                elif len(Wattackers[i-2][j+1])==0:
                    Bbrhorse=1
                         
            Wrthorse = 0
            Brthorse = 0
            if i<7 and j<6 and grid[i+1][j+2]==Empty:
                if len(Battackers[i+1][j+2])==0:
                    Wrthorse=1
                elif len(Wattackers[i+1][j+2])==0:
                    Brthorse=1
                         
            Wrbhorse = 0
            Brbhorse = 0
            if i>0 and j<6 and grid[i-1][j+2]==Empty:
                if len(Battackers[i-1][j+2])==0:
                    Wrbhorse=1
                elif len(Wattackers[i-1][j+2])==0:
                    Brbhorse=1
            
            if current==Brook:
                BrooksDirectionTemp += Bup+Bleft +Bright+Bdown
            elif current==Bknight:
                BknightsDirectionTemp += Btlhorse+Btrhorse+Blthorse+Blbhorse+Bblhorse+Bbrhorse+Brthorse+Brbhorse
                
            elif current==Bbishop:
                BbishopsDirectionTemp += Btldiag+Btrdiag+Bbldiag+Bbrdiag
            
            elif current==Bqueen:
                BqueenDirectionTemp +=Bup+Bleft +Bright+Bdown + Btldiag+Btrdiag+Bbldiag+Bbrdiag
            
            elif current==Bking:
                BkingDirectionTemp += Bup+Bleft +Bright+Bdown + Btldiag+Btrdiag+Bbldiag+Bbrdiag
            
            elif current==Bpawn:
                BpawnsDirectionTemp += Bdown
            
            elif current==Wrook:
                WrooksDirectionTemp += Wup+Wdown+Wleft+Wright
            
            elif current==Wknight:
                WknightsDirectionTemp += Wtlhorse+Wtrhorse+Wlthorse+Wlbhorse+Wblhorse+Wbrhorse+Wrthorse+Wrbhorse
            
            elif current==Wbishop:
                WbishopsDirectionTemp += Wtldiag+Wtrdiag+Wbldiag+Wbrdiag
            
            elif current==Wqueen:
                WqueenDirectionTemp += Wup+Wdown+Wleft+Wright+Wtldiag+Wtrdiag+Wbldiag+Wbrdiag
            
            elif current==Wking:
                WkingDirectionTemp += Wup+Wdown+Wleft+Wright+Wtldiag+Wtrdiag+Wbldiag+Wbrdiag
            
            elif current==Wpawn:
                WpawnsDirectionTemp += Wup
    
    BrooksDirection.append(BrooksDirectionTemp)
    
    BknightsDirection.append(BknightsDirectionTemp)
    
    BbishopsDirection.append(BbishopsDirectionTemp) 
    
    BqueenDirection.append(BqueenDirectionTemp)
    
    BkingDirection.append(BkingDirectionTemp)
    
    BpawnsDirection.append(BpawnsDirectionTemp)
    
    
    WrooksDirection.append(WrooksDirectionTemp)
    
    WknightsDirection.append(WknightsDirectionTemp)
            
    WbishopsDirection.append(WbishopsDirectionTemp)
            
    WqueenDirection.append(WqueenDirectionTemp)
            
    WkingDirection.append(WkingDirectionTemp)
            
    WpawnsDirection.append(WpawnsDirectionTemp)

In [14]:
def countSafeSpaces():
    BsafeTemp = 0
    WsafeTemp = 0
    for i in range(8):
        for j in range(8):
            if i<4:
                current = len(Battackers[i][j])
                if current==0 and grid[i][j]==Empty:
                    WsafeTemp +=1
            else:
                current = len(Wattackers[i][j])
                if current==0 and grid[i][j]==Empty:
                    BsafeTemp +=1
                
    Bsafe.append(BsafeTemp)
    Wsafe.append(WsafeTemp)

In [15]:
def updateScorePST(turn):
    scoreTemp = 0
    for i in range(8):
        for j in range (8):
            current = grid[i][j]
            #pawn, knight, bishop, king
            position = (i*8) + j
            if current==Bknight:
                scoreTemp += -knightTable[position]
                
            elif current==Bbishop:
                scoreTemp += -bishopTable[position]
            
            elif current==Bking:
                if turn>30:
                    scoreTemp += -kingTableLate[position]
                else:
                    scoreTemp += -kingTableEarly[position]
            
            elif current==Bpawn:
                scoreTemp += -pawnTable[position]
            
            elif current==Wknight:
                scoreTemp += knightTable[63-position]
            
            elif current==Wbishop:
                scoreTemp += bishopTable[63-position]
            
            elif current==Wking:
                if turn>30:
                    scoreTemp += kingTableLate[63-position]
                else:
                    scoreTemp +=kingTableEarly[63-position]
            
            elif current==Wpawn:
                scoreTemp += pawnTable[63-position]
                
            #print(scoreTemp)
                
    PSTscore.append(scoreTemp)

In [16]:
def updateValueOfMoves(turn):
    global game_board
    temp_board = copy.deepcopy(game_board)
    moveValues.clear()
    for move in temp_board.legal_moves:
        value = 0
        game_board = copy.deepcopy(temp_board)
        game_board.push(move)
        analyzeState(turn)
        
        #print(game_board)
        #print(" ")
        
        value += BrooksAlive[ len(BrooksAlive)-1] * 10.3386
        value += BknightsAlive[ len(BknightsAlive)-1] * 10.3386
        value += BbishopsAlive[ len(BbishopsAlive)-1] * 10.3386
        value += BqueenAlive[ len(BqueenAlive)-1] * 10.3386
        value += BkingAlive[ len(BkingAlive)-1] * 10.3386
        value += BpawnsAlive[ len(BpawnsAlive)-1] * 10.3386
        value += WrooksAlive[ len(WrooksAlive)-1] * 10.3386
        value += WknightsAlive[ len(WknightsAlive)-1] * 10.3386
        value += WbishopsAlive[ len(WbishopsAlive)-1] * 10.3386
        value += WqueenAlive[ len(WqueenAlive)-1] * 10.3386
        value += WkingAlive[ len(WkingAlive)-1] * 10.3386
        value += WpawnsAlive[ len(WpawnsAlive)-1] * 10.3386
        value += BrooksVulner[ len(BrooksVulner)-1] * 10.3386
        value += BknightsVulner[ len(BknightsVulner)-1] * 10.3386
        value += BbishopsVulner[ len(BbishopsVulner)-1] * 10.3386
        value += BqueenVulner[ len(BqueenVulner)-1] * 10.3386
        value += BkingVulner[ len(BkingVulner)-1] * 10.3386
        value += BpawnsVulner[ len(BpawnsVulner)-1] * 10.3386
        value += WrooksVulner[ len(WrooksVulner)-1] * 10.3386
        value += WknightsVulner[ len(WknightsVulner)-1] * 10.3386
        value += WbishopsVulner[ len(WbishopsVulner)-1] * 10.3386
        value += WqueenVulner[ len(WqueenVulner)-1] * 10.3386
        value += WkingVulner[ len(WkingVulner)-1] * 10.3386
        value += WpawnsVulner[ len(WpawnsVulner)-1] * 10.3386
        value += BrooksDefense[ len(BrooksDefense)-1] * 10.3386
        value += BknightsDefense[ len(BknightsDefense)-1] * 10.3386
        value += BbishopsDefense[ len(BbishopsDefense)-1] * 10.3386
        value += BqueenDefense[ len(BqueenDefense)-1] * 10.3386
        value += BkingDefense[ len(BkingDefense)-1] * 10.3386
        value += BpawnsDefense[ len(BpawnsDefense)-1] * 10.3386
        value += WrooksDefense[ len(WrooksDefense)-1] * 10.3386
        value += WknightsDefense[ len(WknightsDefense)-1] * 10.3386
        value += WbishopsDefense[ len(WbishopsDefense)-1] * 10.3386
        value += WqueenDefense[ len(WqueenDefense)-1] * 10.3386
        value += WkingDefense[ len(WkingDefense)-1] * 10.3386
        value += WpawnsDefense[ len(WpawnsDefense)-1] * 10.3386
        value += BrooksDirection[ len(BrooksDirection)-1] * 10.3386
        value += BknightsDirection[ len(BknightsDirection)-1] * 10.3386
        value += BbishopsDirection[ len(BbishopsDirection)-1] * 10.3386
        value += BqueenDirection[ len(BqueenDirection)-1] * 10.3386
        value += BkingDirection[ len(BkingDirection)-1] * 10.3386
        value += BpawnsDirection[ len(BpawnsDirection)-1] * 10.3386
        value += WrooksDirection[ len(WrooksDirection)-1] * 10.3386
        value += WknightsDirection[ len(WknightsDirection)-1] * 10.3386
        value += WbishopsDirection[ len(WbishopsDirection)-1] * 10.3386
        value += WqueenDirection[ len(WqueenDirection)-1] * 10.3386
        value += WkingDirection[ len(WkingDirection)-1] * 10.3386
        value += WpawnsDirection[ len(WpawnsDirection)-1] * 10.3386
        value += Bsafe[ len(Bsafe)-1] * 10.3386
        value += Wsafe[ len(Wsafe)-1] * 10.3386
        value += PSTscore[ len(PSTscore)-1] * 10.3386
        
        moveValues.append(value)
        
    game_board = copy.deepcopy(temp_board)
        

In [17]:
def chooseTop5():
    top5.clear()
    for i in range(5):
        max_value = max(moveValues)
        max_index = moveValues.index(max_value)
        top5.append(max_index)
        moveValues[max_index] = -9000000000
        

In [18]:
def monte_Carlo(current_board, option_num, turns):
    local_options = current_board.legal_moves
    #print(current_board)
    if current_board.is_game_over():
        #print("it ended")
        if option_num > len(total_outcomes) or option_num > len(win_outcomes):
            print("too large >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        total_outcomes[option_num]+= 1
        victor = 'u'
        result = current_board.result()
        #print(result)
        if result[0] == '0':
            victor = 'b'
        elif result[0] == '1' and result[1] == '-':
            victor = 'w'
        
        if victor == cpu_colour:
            win_outcomes[option_num]+= 1
        #    print("cpu won")
        #else:
        #    print("cpu lost")
        
    else:
        local_temp_board = current_board
        
        
        if not local_temp_board.is_valid():
            print("bad game..........................................................")
        else:
            temp_num = random.randint(0,current_board.legal_moves.count())
            i = 0
            try:
                for local_move in current_board.legal_moves:
                    if i == temp_num:
                        local_temp_board.push(local_move)
                    i+= 1
                monte_Carlo(local_temp_board, option_num, turns+1)
            except IndexError:
                #print("faulty match")
                useless = 1
    return()
    
    

In [19]:
def choose_best():
    options = board.legal_moves
    win_outcomes.clear()
    total_outcomes.clear()
    ratios.clear()
    move_num = 0
    
    for move in board.legal_moves:
        #print("move number is ",move_num)
        #print("move is ",move)
        win_outcomes.append(0)
        total_outcomes.append(0)
        
        if move_num in top5:
            for i in range(iterations_per_option):
                temp = copy.deepcopy(board)
                temp.push(move)
                monte_Carlo(temp, move_num, 1)
                #print(i, end=" ")
                #print("done........................................................")
                #print(" ")
            #print("\n")
        move_num+= 1
    

In [20]:
def AI(turn):
    updateValueOfMoves(turn)
    chooseTop5()
    choose_best()
    ratios.clear()
    for i in range(len(total_outcomes)):
        if total_outcomes[i] == 0:
            ratios.append(0)
        else:
            ratios.append(win_outcomes[i]/total_outcomes[i])
    max_chance = max(ratios)
    max_index = ratios.index(max_chance)
    
    #print(" best option was wiht index ", max_index)
    
    i = 0
    
    for move in board.legal_moves:
        if i == max_index:
            board.push(move)
        i+=1
    

In [21]:
def printOption():
    print(" ")
    for move in board.legal_moves:
        print(move, end=', ')
    print(" ")

In [ ]:
def analyzeState(turn):
    emptyGrid()
    emptyWattackers()
    emptyBattackers()
    updateGridInfo()
    updateWAttackersInfo()
    updateBAttackersInfo()
    countAlive()
    updateVulDefInfo()
    rangeOfMotion()
    countSafeSpaces()
    updateScorePST(turn)

In [ ]:
turns = 0
while not (board.is_game_over()):
    print(board)
    if cpu_colour == 'w':
        game_board = board
        analyzeState(turns)
        print("\n AI is thinking \n")
        AI(turns)
        print("\n AI made move \n")
        print(board)
        printOption()
        player_choice = input("what move would you like to make? ")
        player_move = chess.Move.from_uci(player_choice)
        board.push(player_move)
    else:
        printOption()
        player_choice = input("what move would you like to make? ")
        player_move = chess.Move.from_uci(player_choice)
        board.push(player_move)
        print(" ")
        print(board)
        game_board = board
        analyzeState(turns)
        print("\n AI is thinking \n")
        AI(turns)
        print("\n AI made move \n")
        
    turns+=1
    

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
 
g1h3, g1f3, b1c3, b1a3, h2h3, g2g3, f2f3, e2e3, d2d3, c2c3, b2b3, a2a3, h2h4, g2g4, f2f4, e2e4, d2d4, c2c4, b2b4, a2a4,  
what move would you like to make? g1f3
 
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R

 AI is thinking 


 AI made move 

r n b q k b n r
p p p p p . p p
. . . . . p . .
. . . . . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R
 
f3g5, f3e5, f3h4, f3d4, f3g1, h1g1, b1c3, b1a3, h2h3, g2g3, e2e3, d2d3, c2c3, b2b3, a2a3, h2h4, g2g4, e2e4, d2d4, c2c4, b2b4, a2a4,  
what move would you like to make? f3g5
 
r n b q k b n r
p p p p p . p p
. . . . . p . .
. . . . . . N .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R

 AI is thinking 


 AI made move 

r n b q k b n r
p . p p p . p p
. . . . . p . .
. p . . . . N .
. . . . . 